# GAN





## 1-1 GAN이란?

### 1-1 Vanilla GAN

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import numpy as np
from tqdm.auto import tqdm # 진행상태 확인하는 함수

import matplotlib.pyplot as plt

In [ ]:
!pip install transformers

In [ ]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, )) # -1, 1
])

train_dataset = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST('./data', train=False, transform=transform)

In [ ]:
plt.imshow(train_dataset[0][0].permute(1, 2, 0))

In [ ]:
train_dataset[0][0].shape

학습이 된 Gan모델에 노이즈값을 넣은 뒤 예측한 이미지를 보여주는 함수입니다.

In [ ]:

def plot_generated_images(generator):
    # 100개의 노이즈 값을 생성합니다.
    noise = torch.rand((100, 64), device=device)
    with torch.no_grad():
        make_images = generator(noise).detach().cpu() # 노이즈값을 통해서 GAN으로 이미지 생성합니다.
    make_images = make_images.reshape(100, 1, 28,28) # Mnist 사이즈와 동일하게 바꿔줍니다.
    plt.figure(figsize=(10,10)) # 이미지 크기 지정
    for i in range(make_images.shape[0]): #이미지 갯수 100만큼 반복
        plt.subplot(10, 10, i+1) #바둑판식으로 표현
        plt.imshow(make_images[i].permute(1, 2, 0),plt.get_cmap('gray'))
        plt.axis('off')
    plt.tight_layout()

Vanilla GAN 모델링에 관한 코드입니다.
GAN은 2개의 모델이 필수적으로 사용을 하게 됩니다.

실제이미지와 가짜 이미지를 판별해주는 Discriminator
가짜이미지를 생성하는 Generator 모델입니다.
모델마다 각자의 optimizer를 갖고 있으며 --> 충돌을 방지를 합니다.

데이터는 Fashion Mnist데이터를 이용하여 실습하겠습니다.

In [ ]:
epochs = 20  # 반복 횟수를 정해줍니다.
batch_size = 64 # 배치사이즈를 정해줍니다.

Generator 모델을 만듭니다.
입력데이터는 100개의 노이즈 이며 3층으로 쌓은 FFN 입니다.
활성함수는 ReLU를 사용하였고, 마지막에는 Tanh를 사용하였습니다.

Binary_crossetropy를 사용하여 이진 분류를 사용하였고, optimizer는 아까 규제를 넣은 adam을 사용하였습니다.

Dropout을 사용하게 되었습니다.

Generator의 최종 결과는 784이고 이는 mnist의 이미지 크기와 동일합니다.


In [ ]:
# 생성자
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.generator = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            # nn.Dropout(0.3),
            nn.Linear(512, 784),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.generator(x)
        return x

discriminator 모델을 만듭니다.
입력데이터는 minist파일의 크기이며 3층으로 쌓은 FFN 모델입니다.
Generator와 다르게 unit의 수가 점점 줄어드는 것을 확인 할 수 있습니다.

마지막 활성함수는 sigmoid를 사용함으로서 0 1사이에 거짓 이미지인지 실제 이미지인지를 판별을 해주는 것입니다.

In [ ]:
# 판별자
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.discriminator = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            # nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

        # for param in self.discriminator.parameters():
        #     param.requires_grad = False

    def forward(self, x):
        x = self.discriminator(x)
        return x


이제는 두 모델을 합칩니다.  
실제이미지와 가짜이미지를 판결하기 위해서 Binary_CrossEntropy를 사용하였고,  
input의 데이터는 가짜이미지이고, output은 실제인지 가짜인지 판별해주는 discriminator의 결과값입니다.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_g = Generator()
model_g.to(device)
model_d = Discriminator()
model_d.to(device)

criterion = nn.BCELoss()

#옵티마이저, 손실함수 지정
optimizer_g = optim.Adam(model_g.parameters(), lr=1e-4)
optimizer_d = optim.Adam(model_d.parameters(), lr=1e-4)

scheduler_g = StepLR(optimizer_g, step_size=5, gamma=0.7)
scheduler_d = StepLR(optimizer_d, step_size=5, gamma=0.7)

모델을 학습시키는 코드입니다.

위에서 설정한 Epochs 수만큼 학습을 하게 되고, batch_count만큼 반복하게 됩니다.  

진짜 이미지와 가짜 이미지를 합친 후  
그 크기만큼의 배열을 생성한 뒤 실제 이미지에는 0.9의 값을 주고 가짜 이미지에는 0의 값을 줍니다.  
그 후 discriminator를 학습을 시킵니다.  
배치사이즈 크기만큼 배열을 생성한 뒤 값을 1로 세팅하여 gan에 노이즈값과 함께 학습을 시킵니다.

In [ ]:
# generator가 생성한 이미지를 저장해놓을 리스트입니다.
img_list = []

# 학습에 사용되는 참/거짓의 라벨을 정합니다
real_label = 0.9 # label smoothing = 0.9
fake_label = 0.1

fixed_noise = torch.rand((20, 64), device=device)

total_step = len(train_loader)
for epoch in range(1, epochs + 1):
    for i, (data, _) in enumerate(tqdm(iter(train_loader))):
        # Real 데이터로 discriminator를 학습
        data = data.view(-1, 28*28).to(device)
        # print(data.shape)
        now_batch_size = data.size()[0]

        # 실제 이미지이므로 Discriminator는 1로 판단해야 합니다.
        real_labels = torch.full((now_batch_size, 1), real_label,
                           dtype=torch.float, device=device)
        fake_labels = torch.full((now_batch_size, 1), fake_label,
                           dtype=torch.float, device=device)

        output = model_d(data)
        real_score = output
        d_loss_real = criterion(output, real_labels)

        # Fake 데이터로 discriminator 학습
        noise = torch.randn(now_batch_size, 64).to(device) # 노이즈 데이터를 생성합니다.
        # Generator를 이용해 가짜 이미지를 생성합니다
        # 여기서 generator가 생성한 이미지는 generator에는 영향을 미치지 않기 위해 detach() 합니다.
        fake_data = model_g(noise).detach()
        # print(fake_data.shape)

        # D를 이용해 데이터의 진위를 판별합니다
        output = model_d(fake_data)
        fake_score = output
        # D의 손실값을 계산합니다
        d_loss_fake = criterion(output, fake_labels)

        optimizer_g.zero_grad()
        optimizer_d.zero_grad()
        loss_discriminator = d_loss_real + d_loss_fake
        loss_discriminator.backward()
        optimizer_d.step()

        # Generator를 학습시킵니다.

        noise = torch.randn(now_batch_size, 64).to(device) # 노이즈 데이터를 생성합니다.
        fake_data = model_g(noise)
        output = model_d(fake_data)

        optimizer_g.zero_grad()
        optimizer_d.zero_grad()
        loss_generator = criterion(output, real_labels)
        loss_generator.backward()
        # G를 업데이트 합니다
        optimizer_g.step()

        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'
                  .format(epoch, epochs, i+1, total_step, loss_discriminator.item(), loss_generator.item(),
                          real_score.mean().item(), fake_score.mean().item()))

    scheduler_g.step()
    scheduler_d.step()

    with torch.no_grad():
        fake = model_g(fixed_noise).detach().cpu()

    make_images = fake.reshape(20, 1, 28,28) # Mnist 사이즈와 동일하게 바꿔줍니다.
    img_list.append(make_images)


In [ ]:
model_g

In [ ]:
img_list[0]

In [ ]:

make_images.shape[0]

In [ ]:
i

In [ ]:
make_images.shape

In [ ]:
len(img_list)

In [ ]:
for make_images in img_list:
    if make_images.shape[1] != 784:
        plt.figure(figsize=(4,4)) # 이미지 크기 지정
        for i in range(make_images.shape[0]): #이미지 갯수만큼 반복
            plt.subplot(4, 5, i+1) #바둑판식으로 표현
            plt.imshow(make_images[i].permute(1, 2, 0))
            plt.axis('off')
        plt.show()

In [ ]:
plot_generated_images(model_g)

### 1-2 DCGAN

이번에는 GAN의 응용모델인 DCGAN을 설명하겠습니다.  
DCGAN은 "Deep Convolutional Generative Adversarial Nets"의 약자이며 FFN 부분을 CNN으로 바꾼 부분입니다.  
DCGAN이 탄생하게 된 것은 기존에 있던 GAN이 불안정하며 새로 만들어진 결과물들이 좋은지를 판단하기가 어렵게 되어 나오게 된 것입니다.  
기존에 있던 CNN과 다른 점은 Pooling layer를 사용하지 않고 이를 strides로 교체를 한 것입니다.

Vanilla GAN을 했을 당시에는 -1 ~ 1 사이의 값으로 하였으나
DCGAN에서는 결과가 좋지 않아 0 ~ 1 사이의 값으로 정규화를 진행하였습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html

In [ ]:
import zipfile

house_zip = zipfile.ZipFile('/content/drive/MyDrive/머신러닝/data/img_align_celeba.zip')
house_zip.extractall('./data/celeba')
house_zip.close()

In [ ]:
dataroot = "./data/celeba"

In [ ]:
face_dataset = datasets.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(64),
                               transforms.CenterCrop(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

In [ ]:
batch_size=128
dataloader = torch.utils.data.DataLoader(face_dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=0)

Generator를 선언하는 부분입니다.  
해당 코드에서 중요한 것은 Strides 부분입니다.  
처음 보는 함수들이 있습니다.

LeakyReLU는 활성함수 중 하나입니다.  
Relu는 0이하인 값들은 전부 0으로 바꿔주지만 LeakyReLU는 0으로 수렴하게 해줍니다.  
Conv2DTranspose라는 Deconvolution라고 불리우고 이미지 해상도를 올리 때 사용합니다.  
Conv2D와 사용법은 동일하고 Conv2D를 거꾸로 해놓은 버전이라고 생각하시면 됩니다.

처음 FFN으로 통하여 이미지의 크기를 지정해줍니다.  
그 후 리사이즈를 통해 이미지가 Conv2D에 학습 할 수 있도록 바꿔줍니다.

Conv2DTranspose에서 필터사이즈 256, 커널사이즈 4, strides 2, padding은 same으로 설정 하였습니다.  
다층 Conv2D를 지나고 Dense로 가게 되면 28  28  1 이미지의 형태가 됩니다.

Summary 부분에서 자세하게 설명하겠습니다.

In [ ]:
# 생성자
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.generator = nn.Sequential(
            nn.ConvTranspose2d(64, 64*8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64*8),
            nn.ReLU(),

            nn.ConvTranspose2d(64*8, 64*4, kernel_size=4, stride=2, padding=1), # 같은 크기로 Conv2D
            nn.BatchNorm2d(64*4),
            nn.ReLU(),

            nn.ConvTranspose2d(64*4, 64*2, kernel_size=4, stride=2, padding=1), # Deconvolution
            nn.BatchNorm2d(64*2),
            nn.ReLU(),

            nn.ConvTranspose2d(64*2, 64*1, kernel_size=4, stride=2, padding=1), # 같은 크기로 Conv2D
            nn.BatchNorm2d(64*1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1), # 같은 크기로 Conv2D
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.generator(x)
        return x


In [ ]:
import torchsummary
torchsummary.summary(Generator().cuda(), (64, 1, 1, ))

Summary를 살펴 보게 되면  
첫 Conv2D를 지나게 되면 14 * 14 * 256의 이미지 형태가 됩니다.  
Conv2DTranspose 부분에서 지나게 되면서 28 * 28 * 256 형태로 바뀌게 됩니다.  
이는 strides부분이 기존에 있던 크기를 확장시키게 됩니다.  
strides가 3이면 이미지는 42 * 42 * 256 형태가 되는 것입니다.  
padding을 설정하게 됨으로써 strides 부분만 영향을 주게 되는 것입니다.

위에 코드에서 strides와 padding을 조정하고 summary를 출력을 해보면 이해하기 쉬울 것입니다.

Discriminator는 FFN 부분이 Conv2D로 바뀐 말고는 달라진 점은 없습니다.

Conv2D 코드를 보면 strides 부분이 추가가 되어있으며 Polling 부분이 없는 것이 확인 됩니다.

Adam이 아닌 RMSprop으로 진행하였습니다.

In [ ]:
# 생성자
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.discriminator = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.4),
            nn.Conv2d(64, 64*2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.4),
            nn.Conv2d(64*2, 64*4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64*4, 64*8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Flatten(),
            # nn.Dropout(p=0.4),
            nn.Linear(64*8*4*4, 1),
            nn.Sigmoid(),

        )

    def forward(self, x):
        x = self.discriminator(x)
        return x


Summary를 살펴보면 Padding이 설정되지 않기 대문에 커널 사이즈의 영향을 받아 이미지 크기가 줄어 들었습니다.  
또한 strides 부분이 2로 설정 되어있있기에 이미지의 2번 째 Conv2D 부터는 이미지 크기가 줄어드는 게 보입니다.

In [ ]:
torchsummary.summary(Discriminator().cuda(), (3, 64, 64))

학습을 시키는 부분은 GAN 기본 모델과 다르지 않습니다.

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_g = Generator()
model_g.to(device)
model_d = Discriminator()
model_d.to(device)

# for para in model_d.parameters():
#     para.requires_grad = False

criterion = nn.BCELoss().cuda()
optimizer_g = optim.Adam(model_g.parameters(), lr=2e-3)
optimizer_d = optim.Adam(model_d.parameters(), lr=2e-3)
scheduler_g = StepLR(optimizer_g, step_size=5, gamma=0.7)
scheduler_d = StepLR(optimizer_d, step_size=5, gamma=0.7)

In [ ]:
# generator가 생성한 이미지를 저장해놓을 리스트입니다.
img_list = []

# 학습에 사용되는 참/거짓의 라벨을 정합니다
real_label = 0.9 # label smoothing = 0.9
fake_label = 0.

fixed_noise = torch.rand((20, 64, 1, 1), device=device)

epochs = 5
total_step = len(dataloader)

for epoch in range(1, epochs + 1):
    for i, (data, _) in enumerate(tqdm(iter(dataloader))):
        # Real 데이터로 discriminator를 학습
        data = data.to(device)
        # print(data.shape)
        now_batch_size = data.size()[0]

        # 실제 이미지이므로 Discriminator는 1로 판단해야 합니다.
        real_labels = torch.full((now_batch_size, 1), real_label,
                           dtype=torch.float, device=device)
        fake_labels = torch.full((now_batch_size, 1), fake_label,
                           dtype=torch.float, device=device)

        output = model_d(data)
        real_score = output
        d_loss_real = criterion(output, real_labels)

        # Fake 데이터로 discriminator 학습
        noise = torch.randn(now_batch_size, 64, 1, 1).to(device) # 노이즈 데이터를 생성합니다.
        # Generator를 이용해 가짜 이미지를 생성합니다
        fake_data = model_g(noise).detach()
        # print(fake_data.shape)

        # D를 이용해 데이터의 진위를 판별합니다
        output = model_d(fake_data)
        fake_score = output
        # D의 손실값을 계산합니다
        d_loss_fake = criterion(output, fake_labels)

        optimizer_g.zero_grad()
        optimizer_d.zero_grad()
        loss_discriminator = d_loss_real + d_loss_fake
        loss_discriminator.backward()
        optimizer_d.step()

        # Generator를 학습시킵니다.

        noise = torch.randn(now_batch_size, 64, 1, 1).to(device) # 노이즈 데이터를 생성합니다.
        fake_data = model_g(noise)
        output = model_d(fake_data)

        optimizer_g.zero_grad()
        optimizer_d.zero_grad()
        loss_generator = criterion(output, real_labels)
        loss_generator.backward()
        # G를 업데이트 합니다
        optimizer_g.step()

        if (i+1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'
                  .format(epoch, epochs, i+1, total_step, loss_discriminator.item(), loss_generator.item(),
                          real_score.mean().item(), fake_score.mean().item()))

    scheduler_g.step()
    scheduler_d.step()

    with torch.no_grad():
        fake = model_g(fixed_noise).detach().cpu()

    img_list.append(fake)


In [ ]:
while True:pass

In [ ]:
make_images[i][0]

In [ ]:
denorm = transforms.Compose([
                            transforms.Normalize((-1 * 0.5 / 0.5, -1 * 0.5 / 0.5, -1 * 0.5 / 0.5), (1.0 / 0.5, 1.0 / 0.5, 1.0 / 0.5)),
                           ])

In [ ]:
denorm(make_images[i])

In [ ]:
for make_images in img_list:
    # if make_images.shape[1] != 784:
    plt.figure(figsize=(4,4)) # 이미지 크기 지정
    for i in range(make_images.shape[0]): #이미지 갯수만큼 반복
        plt.subplot(4, 5, i+1) #바둑판식으로 표현
        plt.imshow(denorm(make_images[i]).permute(1, 2, 0))
        plt.axis('off')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
def plot_generated_images(generator):
    # 100개의 노이즈 값을 생성합니다.
    noise = torch.rand((100, 64, 1, 1), device=device)
    with torch.no_grad():
        make_images = generator(noise).detach().cpu() # 노이즈값을 통해서 GAN으로 이미지 생성합니다.
    make_images = make_images.reshape(100, 3, 64, 64) # Mnist 사이즈와 동일하게 바꿔줍니다.
    plt.figure(figsize=(10,10)) # 이미지 크기 지정
    for i in range(make_images.shape[0]): #이미지 갯수 100만큼 반복
        plt.subplot(10, 10, i+1) #바둑판식으로 표현
        plt.imshow(denorm(make_images[i]).permute(1, 2, 0))
        plt.axis('off')
    plt.tight_layout()

In [ ]:
plot_generated_images(model_g)